# Transformer-based Sentiment Analysis (BERT)

## Objectif
- Introduire un modèle Transformer (BERT)
- Tester une approche contextualisée du sentiment
- Comparer qualitativement avec TF-IDF + SVM

Ce notebook est proposé comme une ouverture vers l’état de l’art.


In [13]:
%pip install transformers torch datasets accelerate -q

Note: you may need to restart the kernel to use updated packages.


In [14]:
import pandas as pd
import torch

from transformers import (
    BertTokenizer,
    BertForSequenceClassification,
    Trainer,
    TrainingArguments
)


In [15]:
df = pd.read_csv("../data/feedbacks_enriched.csv")

df = df[["clean_text", "rating_sentiment"]]
df.head()


,clean_text,rating_sentiment
0,nice hotel expensive parking got good deal sta...,positive
1,nothing special charge diamond member hilton d...,negative
2,nice room experience hotel monaco seattle good...,neutral
3,unique great stay wonderful time hotel monaco ...,positive
4,great stay great stay went seahawk game awesom...,positive


In [16]:
label2id = {
    "negative": 0,
    "neutral": 1,
    "positive": 2
}
id2label = {v: k for k, v in label2id.items()}

df["label"] = df["rating_sentiment"].map(label2id)


In [17]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(
    df,
    test_size=0.2,
    random_state=42,
    stratify=df["label"]
)


In [18]:
model_name = "bert-base-uncased"

tokenizer = BertTokenizer.from_pretrained(model_name)
def tokenize_text(texts):
    return tokenizer(
        texts.tolist(),
        padding=True,
        truncation=True,
        max_length=128,
        return_tensors="pt"
    )


In [19]:
#Dataset PyTorch minimal
class FeedbackDataset(torch.utils.data.Dataset):
    def __init__(self, texts, labels):
        self.encodings = tokenize_text(texts)
        self.labels = torch.tensor(labels.tolist())

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item["labels"] = self.labels[idx]
        return item
train_dataset = FeedbackDataset(train_df["clean_text"], train_df["label"])
test_dataset = FeedbackDataset(test_df["clean_text"], test_df["label"])

In [20]:
#bert model
model = BertForSequenceClassification.from_pretrained(
    model_name,
    num_labels=3,
    id2label=id2label,
    label2id=label2id
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [21]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./bert_results",
    overwrite_output_dir=True,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=2,
    learning_rate=2e-5,
    weight_decay=0.01,
    logging_steps=50,
    save_steps=10000,
    save_total_limit=1
)

In [22]:
# Limit samples to avoid RAM issues
n_samples = 50  # change to None for full dataset

if n_samples is not None:
    train_df_small = train_df.iloc[:n_samples].reset_index(drop=True)
    test_df_small = test_df.iloc[:n_samples].reset_index(drop=True)
else:
    train_df_small = train_df
    test_df_small = test_df

print("Train size:", len(train_df_small))
print("Test size:", len(test_df_small))

Train size: 50
Test size: 50


In [23]:
train_dataset = FeedbackDataset(
    train_df_small["clean_text"],
    train_df_small["label"]
)

test_dataset = FeedbackDataset(
    test_df_small["clean_text"],
    test_df_small["label"]
)


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

trainer.train()


/home/sakkarouis/anaconda3/envs/nlp_env/lib/python3.11/site-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


In [ ]:
trainer.train()

/home/sakkarouis/anaconda3/envs/nlp_env/lib/python3.11/site-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss


TrainOutput(global_step=14, training_loss=0.8769009453909737, metrics={'train_runtime': 93.8784, 'train_samples_per_second': 1.065, 'train_steps_per_second': 0.149, 'total_flos': 6577835443200.0, 'train_loss': 0.8769009453909737, 'epoch': 2.0})

In [ ]:
predictions = trainer.predict(test_dataset)
y_pred = predictions.predictions.argmax(axis=1)


/home/sakkarouis/anaconda3/envs/nlp_env/lib/python3.11/site-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


In [ ]:
from sklearn.metrics import classification_report

y_true = test_df_small["label"].values  # ✅ 50 labels

print(
    classification_report(
        y_true,
        y_pred,
        target_names=["negative", "neutral", "positive"]
    )
)


              precision    recall  f1-score   support

    negative       0.00      0.00      0.00         3
     neutral       0.00      0.00      0.00         9
    positive       0.76      1.00      0.86        38

    accuracy                           0.76        50
   macro avg       0.25      0.33      0.29        50
weighted avg       0.58      0.76      0.66        50



/home/sakkarouis/anaconda3/envs/nlp_env/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1833: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/sakkarouis/anaconda3/envs/nlp_env/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1833: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/sakkarouis/anaconda3/envs/nlp_env/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1833: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifi

In [ ]:
# Use ONLY the reduced test dataframe (50 samples)
test_df_small = test_df_small.copy()

test_df_small["bert_pred"] = [id2label[i] for i in y_pred]

test_df_small[[
    "clean_text",
    "rating_sentiment",
    "bert_pred"
]].head(10)


NameError: name 'test_df_small' is not defined

## Conclusion

Cette expérimentation montre que :
- BERT permet de capturer le contexte sémantique des feedbacks
- Les performances peuvent être comparables ou supérieures
- Le coût computationnel est nettement plus élevé

Dans le cadre de ce projet, BERT est présenté comme une ouverture
vers l’état de l’art plutôt qu’une solution de production.
